In [ ]:
from PIL import Image
import json

def convert_image_to_led_matrix_format(image_path):
    # Open the image
    img = Image.open(image_path)
    
    # Ensure the image is in RGB format
    img = img.convert('RGB')
    
    # Get image dimensions
    width, height = img.size
    
    # Initialize the list to store pixel values
    pixel_data = []
    
    # Iterate over each pixel
    for y in range(height):
        for x in range(width):
            # Get RGB values
            R, G, B = img.getpixel((x, y))
            # Append the dictionary with pixel details to the list
            pixel_data.append({
                'x': x,
                'y': y,
                'R': R,
                'G': G,
                'B': B
            })
    
    return pixel_data

# Example usage
image_path = '/home/qthi/Documents/quyth/Image/64x64/img1.png'
led_matrix_data = convert_image_to_led_matrix_format(image_path)

# Convert the list of pixel data to JSON format
led_matrix_data_json = json.dumps(led_matrix_data, indent=4)

# Print the JSON result
print(led_matrix_data_json)

# Optionally, save the JSON data to a file
with open('led_matrix_data.json', 'w') as json_file:
    json_file.write(led_matrix_data_json)


In [1]:
from PIL import Image
import json
import zlib
import base64

# Đọc hình ảnh và thay đổi kích thước thành 64x64
image = Image.open('/home/qthi/Documents/quyth/Image/64x64/img1.png')
image = image.resize((64, 64))
image = image.convert('RGB')

# Lấy dữ liệu điểm ảnh
pixels = list(image.getdata())
width, height = image.size

# Tạo mảng điểm ảnh với định dạng (x, y, R, G, B)
pixel_data = []
for y in range(height):
    for x in range(width):
        r, g, b = pixels[y * width + x]
        pixel_data.append({"x": x, "y": y, "R": r, "G": g, "B": b})

# Chuyển đổi mảng điểm ảnh thành JSON
pixel_data_json = json.dumps(pixel_data)

# Nén dữ liệu JSON
compressed_data = zlib.compress(pixel_data_json.encode('utf-8'))

# Mã hóa dữ liệu nén bằng Base64
encoded_data = base64.b64encode(compressed_data).decode('utf-8')

# Lưu dữ liệu Base64 vào file (tùy chọn)
with open('encoded_pixel_data.txt', 'w') as f:
    f.write(encoded_data)


In [7]:
from PIL import Image
import struct
import asyncio
import websockets
# Đọc hình ảnh và thay đổi kích thước thành 64x64
image = Image.open('/home/qthi/Documents/quyth/Image/64x64/img1.png')
image = image.resize((64, 64))
image = image.convert('RGB')

# Lấy dữ liệu điểm ảnh
pixels = list(image.getdata())
width, height = image.size

# Tạo dữ liệu nhị phân
binary_data = bytearray()
for y in range(height):
    for x in range(width):
        r, g, b = pixels[y * width + x]
        binary_data.extend(struct.pack('BBBBB', x, y, r, g, b))

# Lưu dữ liệu nhị phân vào tệp
with open('pixel_data.bin', 'wb') as file:
    file.write(binary_data)

print("Dữ liệu nhị phân đã được lưu vào 'pixel_data.bin'")

async def send_data():
    uri = "ws://192.168.1.189:80/ws"
    async with websockets.connect(uri) as websocket:
        # Đọc dữ liệu nhị phân từ tệp
        with open('pixel_data.bin', 'rb') as file:
            binary_data = file.read()
        
        # Gửi dữ liệu nhị phân
        await websocket.send(binary_data)
        print("Dữ liệu nhị phân đã được gửi")

send_data()


Dữ liệu nhị phân đã được lưu vào 'pixel_data.bin'


<coroutine object send_data at 0x7fd2c09be8c0>

In [15]:
send_data()

<coroutine object send_data at 0x7fd2c09bed40>

In [ ]:
import asyncio
import websockets
import struct

async def send_binary_data(uri):
    async with websockets.connect(uri) as websocket:
        # Example: Create binary data representing pixels
        width, height = 64, 64
        binary_data = bytearray()

        for y in range(height):
            for x in range(width):
                r, g, b = (255, 0, 0)  # Example: Red color for all pixels
                binary_data.extend(struct.pack('BBBBB', x, y, r, g, b))

        # Send binary data
        await websocket.send(binary_data)
        print("Binary data sent to the server")

        # Optionally, receive a response from the server
        response = await websocket.recv()
        print(f"Received from server: {response}")

# Replace <ESP8266_IP> with the actual IP address of your ESP8266
uri = "ws://192.168.1.189:80/ws"

# Run the client
asyncio.get_event_loop().run_until_complete(send_binary_data(uri))


In [ ]:
import websocket

ws = websocket.WebSocket()
ws.connect("ws://192.168.1.189:80/ws")

ws.send_binary([100, 220, 130])

ws.close()

In [2]:
from PIL import Image

def image_to_array(image_path):
    # Load the image
    img = Image.open(image_path)
    
    # Convert the image to 1-bit color (black and white)
    img = img.convert('1')
    
    # Get image dimensions
    width, height = img.size
    
    # Prepare array to hold the image data
    img_array = []
    
    # Iterate over the image pixels
    for y in range(height):
        byte = 0
        for x in range(width):
            # Get the pixel value (0 or 1)
            pixel = img.getpixel((x, y))
            
            # Set the corresponding bit in the byte
            byte |= (pixel == 0) << (7 - (x % 8))
            
            # Every 8 bits, append the byte to the array
            if x % 8 == 7 or x == width - 1:
                img_array.append(byte)
                byte = 0
    
    return img_array, width, height

def format_array_as_c(array, width, height):
    rows = []
    for i in range(0, len(array), (width + 7) // 8):
        row = array[i:i + (width + 7) // 8]
        row_str = ','.join(f'0x{byte:02x}' for byte in row)
        rows.append(f"  {row_str}")
    
    formatted_str = "const char wifi_image1bit[] PROGMEM = {\n" + ",\n".join(rows) + "\n};"
    return formatted_str

def save_array_to_file(array_str, file_path):
    with open(file_path, 'w') as f:
        f.write(array_str)

# Example usage
image_path = '/home/qthi/Documents/quyth/Image/64x64/img1.png'  # Replace with the path to your image
output_file_path = 'output_array.c'  # Replace with the desired output file path

array, width, height = image_to_array(image_path)
formatted_array = format_array_as_c(array, width, height)
save_array_to_file(formatted_array, output_file_path)

print(f"C array has been saved to {output_file_path}")


C array has been saved to output_array.c


In [1]:
from PIL import Image

def image_to_pixel_data(image_path):
    """Convert an image to a 2D array of pixel data in the format [[x, y, R, G, B], ...]"""
    # Open the image
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        width, height = img.size
        pixel_data = []

        # Extract pixel data
        for y in range(height):
            for x in range(width):
                r, g, b = img.getpixel((x, y))
                pixel_data.append([x, y, r, g, b])  # Append each pixel data as a list
        
    return pixel_data

def save_pixel_data_to_file(pixel_data, file_path):
    """Save the pixel data to a file"""
    with open(file_path, 'w') as file:
        for pixel in pixel_data:
            file.write(','.join(map(str, pixel)) + ',\n')

# Example usage
image_path = '/home/qthi/Documents/quyth/Image/64x64/img1.png'
pixel_data = image_to_pixel_data(image_path)

# Save pixel data to a file
file_path = 'pixel_data1.txt'
save_pixel_data_to_file(pixel_data, file_path)

print(f"Pixel data saved to {file_path}")


Pixel data saved to pixel_data1.txt


In [ ]:
import requests
import json

url = 'http://192.168.1.56/imgbit1'
chunk_size = 1024  # Adjust as needed

def send_chunks(data, url):
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        response = requests.post(url, data=json.dumps(chunk), headers={'Content-Type': 'application/json'})
        print(response.text)

# Prepare data
data = [[0]*5 for _ in range(4096)]  # Replace with actual data
serialized_data = json.dumps(data)

# Send data in chunks
send_chunks(serialized_data, url)


In [1]:
from PIL import Image

def rgb888_to_rgb565(r, g, b):
    """Convert 8-bit RGB values to 16-bit RGB565."""
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)

def image_to_rgb565_array(image_path):
    """Convert an image to a list of RGB565 values."""
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        width, height = img.size
        pixel_data = []

        for y in range(height):
            for x in range(width):
                r, g, b = img.getpixel((x, y))
                rgb565 = rgb888_to_rgb565(r, g, b)
                pixel_data.append(rgb565)
        
    return pixel_data, width, height

# Example usage
image_path = '/home/qthi/Documents/quyth/Image/64x64/img1.png'
pixel_data, width, height = image_to_rgb565_array(image_path)

# Save to a file (optional)
with open('rgb565_data.h', 'w') as file:
    file.write(f'const uint16_t image_data[{width * height}] = {{\n')
    for i, value in enumerate(pixel_data):
        if i % width == 0:
            file.write('\n')
        file.write(f'0x{value:04X}, ')
    file.write('\n};\n')

print(f"RGB565 data saved to 'rgb565_data.h'.")


RGB565 data saved to 'rgb565_data.h'.


In [4]:
import socket
from PIL import Image

def rgb888_to_rgb565(r, g, b):
    """Convert 8-bit RGB values to 16-bit RGB565."""
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)

def image_to_rgb565_array(image_path):
    """Convert an image to a list of RGB565 values."""
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        width, height = img.size
        pixel_data = []

        for y in range(height):
            for x in range(width):
                r, g, b = img.getpixel((x, y))
                rgb565 = rgb888_to_rgb565(r, g, b)
                pixel_data.append(rgb565)
        
    return pixel_data, width, height

# Convert image to RGB565 array
image_path = '/home/qthi/Documents/quyth/Image/64x64/img2.png'
pixel_data, width, height = image_to_rgb565_array(image_path)

# UDP settings
UDP_IP = "192.168.1.112"
UDP_PORT = 5000

# Prepare data for UDP
data = bytearray()
for pixel in pixel_data:
    data.extend(pixel.to_bytes(2, 'big'))

# Send data via UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.sendto(data, (UDP_IP, UDP_PORT))
print(f"Sent {len(data)} bytes to {UDP_IP}:{UDP_PORT}")

Sent 8192 bytes to 192.168.1.112:5000


In [13]:
from math import sin, pi, sqrt
import math
import struct

def generate_plasma(width, height,t):
    plasma = []
    
    for y in range(height):
        for x in range(width):
            color = int((
                int((128.0 + (128.0 * sin(x / 16.0))) +
                    (128.0 + (128.0 * sin(y / 8.0))) +
                    (128.0 + (128.0 * sin((x + y) / 16.0))) +
                    (128.0 + (128.0 * sin(sqrt(x * x + y * y) / 8.0 + t)))) / 4
            ))
            rgb565_1 = rgb888_to_rgb565(color, color, color)
            plasma.append(rgb565_1)
    print(plasma)

In [14]:
t = 0
generate_plasma(64,64,t)
t+=1

[33808, 33840, 35953, 38066, 40179, 42292, 44405, 46486, 48599, 48631, 50712, 52825, 52825, 52857, 54938, 54938, 54938, 54938, 54938, 54938, 52857, 52857, 52825, 50744, 50744, 50712, 48631, 48599, 46486, 44405, 44373, 42292, 42260, 40179, 40147, 38066, 38034, 35953, 35953, 35921, 33840, 33840, 33840, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 31727, 31727, 31727, 31727, 31727, 31727, 31727, 31727, 31695, 31695, 29614, 29614, 29582, 35921, 35953, 38066, 40179, 42292, 44405, 46486, 48599, 48631, 50744, 52825, 52857, 54938, 54938, 54970, 54970, 54970, 54970, 54970, 54970, 54970, 54938, 52857, 52825, 52825, 50744, 50712, 48599, 46518, 46486, 44405, 44373, 42292, 42260, 40179, 40147, 38066, 38034, 35953, 35953, 35921, 35921, 33840, 33840, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 33808, 31727, 31727, 31727, 31695, 31695, 29614, 29614, 38034, 40147, 40179, 42292, 44405, 46486, 48599, 48631, 50744, 52825, 52857, 54938, 54970, 57051, 57051

In [8]:
from PIL import Image
def image_to_rgb565_array(image_path):
    """Convert an image to a list of RGB565 values."""
    with Image.open(image_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        width, height = img.size
        pixel_data = []

        for y in range(height):
            for x in range(width):
                r, g, b = img.getpixel((x, y))
                rgb565 = rgb888_to_rgb565(r, g, b)
                pixel_data.append(rgb565)
    print(pixel_data) 
def rgb888_to_rgb565(r, g, b):
    r = (r >> 3) & 0x1F
    g = (g >> 2) & 0x3F
    b = (b >> 3) & 0x1F
    return (r << 11) | (g << 5) | b


In [9]:
image_to_rgb565_array('/home/qthi/Documents/quyth/Image/64x64/img2.png')

[61342, 61310, 61342, 63422, 63391, 61342, 61342, 61342, 61342, 63390, 61342, 61342, 61342, 63390, 61342, 61342, 61342, 61342, 63423, 61342, 61310, 61310, 61310, 61310, 61310, 61310, 61310, 61310, 61342, 61310, 61310, 61310, 61342, 61309, 61309, 61310, 61309, 61309, 59262, 59262, 59229, 59229, 59229, 59229, 59229, 59197, 61310, 59229, 57149, 57116, 59197, 59229, 59197, 57148, 57149, 59197, 57116, 59196, 57116, 57116, 59197, 57084, 57084, 57084, 61342, 63422, 61342, 61342, 63422, 61342, 61342, 61342, 61342, 61342, 61310, 61342, 61342, 61342, 61342, 61342, 61342, 61342, 63422, 61310, 61310, 61310, 61310, 61310, 61310, 61310, 61342, 61342, 61309, 61342, 61342, 61310, 63423, 61310, 59229, 61310, 61310, 59229, 59261, 59262, 59197, 59229, 59229, 59229, 59229, 59229, 61310, 59197, 57149, 57149, 59197, 59229, 57148, 59197, 57149, 59197, 57148, 57116, 59165, 57116, 59197, 57116, 57084, 57084, 63423, 61342, 61342, 63423, 63390, 63390, 61342, 61342, 63422, 63422, 63423, 61342, 61342, 63391, 63423